In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import glob
from census import Census
from us import states

import os
from census import Census
from pprint import pprint

# Census API Key
from config import api_key

buz_info  = 44021

# zipcode captured from:  https://www.zipcodestogo.com/New%20York/ 
zipcode = pd.read_csv("ny_city_zipcodes.csv")
zipcode.head()

,Zip Code,City,County,Zip Code Map
0,501,Holtsville,Suffolk,View Map
1,544,Holtsville,Suffolk,View Map
2,6390,Fishers Island,Suffolk,View Map
3,10001,New York,New York,View Map
4,10002,New York,New York,View Map


In [3]:
zipcode.columns

Index(['Zip Code', 'City', 'County', 'Zip Code Map'], dtype='object')

In [4]:
# test =zipcode.filter(like='New York', axis=0)

array = ['Richmond','Bronx','Queens','Kings','New York']
City_zip = zipcode.loc[zipcode['County'].isin(array)]
City_zip.head()

,Zip Code,City,County,Zip Code Map
3,10001,New York,New York,View Map
4,10002,New York,New York,View Map
5,10003,New York,New York,View Map
6,10004,New York,New York,View Map
7,10005,New York,New York,View Map


In [5]:
zcode = City_zip['Zip Code']

In [6]:
print(len(zcode))

333


In [7]:
list(zcode)

[10001,
 10002,
 10003,
 10004,
 10005,
 10006,
 10007,
 10008,
 10009,
 10010,
 10011,
 10012,
 10013,
 10014,
 10015,
 10016,
 10017,
 10018,
 10019,
 10020,
 10021,
 10022,
 10023,
 10024,
 10025,
 10026,
 10027,
 10028,
 10029,
 10030,
 10031,
 10032,
 10033,
 10034,
 10035,
 10036,
 10037,
 10038,
 10039,
 10040,
 10041,
 10043,
 10044,
 10045,
 10046,
 10047,
 10048,
 10055,
 10060,
 10069,
 10072,
 10079,
 10080,
 10081,
 10082,
 10087,
 10090,
 10094,
 10095,
 10096,
 10098,
 10099,
 10101,
 10102,
 10103,
 10104,
 10105,
 10106,
 10107,
 10108,
 10109,
 10110,
 10111,
 10112,
 10113,
 10114,
 10115,
 10116,
 10117,
 10118,
 10119,
 10120,
 10121,
 10122,
 10123,
 10124,
 10125,
 10126,
 10128,
 10129,
 10130,
 10131,
 10132,
 10133,
 10138,
 10149,
 10150,
 10151,
 10152,
 10153,
 10154,
 10155,
 10156,
 10157,
 10158,
 10159,
 10160,
 10161,
 10162,
 10163,
 10164,
 10165,
 10166,
 10167,
 10168,
 10169,
 10170,
 10171,
 10172,
 10173,
 10174,
 10175,
 10176,
 10177,
 10178,


In [21]:
# set up a parameters dictionary
ZIPCODE = []
PAYANN = []
ziptable = []

years = [2012,2013,2014,2015,2016]




for year in years:
    for code in zcode:  

        # base url
        query_url = f"https://api.census.gov/data/{year}/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:{code}&key={api_key}"

        print(query_url) 

        #response = requests.get(base_url).json()


        #Run requests to grab the JSON at the requested URL

        response = requests.get(query_url)
        print(response.status_code)

        if response.status_code == 200:
            jsonResponse = response.json()
            ZIPCODE.append(jsonResponse[1][0])       
            PAYANN.append(jsonResponse[1][2]) 

    NewYork_payfoll_by_ZIP_df = pd.DataFrame({"Year":year,"ZIPCODE": ZIPCODE,"Total Annual Payroll": PAYANN})
    NewYork_payfoll_by_ZIP_df.head()

    c = Census(api_key, year=year)
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                              "B19301_001E",
                              "B17001_002E","B02001_002E"), {'for': 'zip code tabulation area:*'})

    # Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

    # Column Reordering
    census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                          "B01002_001E": "Median Age",
                                          "B19013_001E": "Household Income",
                                          "B19301_001E": "Per Capita Income",
                                          "B17001_002E": "Poverty Count",
                                          "B02001_002E": "Whites Alone",
                                          "NAME": "Name", "zip code tabulation area": "Zipcode"})

    # Add in Poverty Rate (Poverty Count / Population)
    census_pd["Poverty Rate"] = 100 * \
        census_pd["Poverty Count"].astype(
            int) / census_pd["Population"].astype(int)

    # Add in none white (whites alone Count - Population)
    census_pd["Non Whites"] =  \
        census_pd["Population"].astype(
            int) - census_pd["Whites Alone"].astype(int)

    # Final DataFrame
    census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                           "Per Capita Income", "Poverty Count", "Poverty Rate","Whites Alone", "Non Whites"]]

    # Rename zipcode column without creating new dataframe
    census_pd.rename(columns={'Zipcode': 'ZIPCODE'}, inplace=True)
    
    a=year
    
    # Merge two dataframes using an inner join
    df = pd.merge(NewYork_payfoll_by_ZIP_df, census_pd, on=["ZIPCODE"])
    ziptable.append(df)



#df=pd.merge(NewYork.....)
#     all_df.append(df）
merged_df=pd.concat(ziptable)

https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10001&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10002&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10003&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10004&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10005&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10006&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10007&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10008&k

https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10102&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10103&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10104&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10105&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10106&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10107&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10108&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10109&k

204
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10185&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10196&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10197&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10199&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10203&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10211&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10212&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:102

200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10465&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10466&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10467&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10468&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10469&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10470&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10471&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:104

https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11243&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11244&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11245&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11247&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11248&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11249&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11251&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2012/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11252&k

...

204
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10095&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10096&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10098&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10099&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10101&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10102&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10103&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:101

200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10175&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10176&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10177&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10178&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10179&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10184&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10185&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:101

200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10459&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10460&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10461&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10462&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10463&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10464&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:10465&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:104

https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11237&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11238&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11239&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11240&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
204
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11241&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11242&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11243&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11244&k

200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11424&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11425&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11426&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11427&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11428&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11429&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:11430&key=2af5575ace0b8385c92d40ea1d52fe11407acc24
200
https://api.census.gov/data/2016/zbp?get=ZIPCODE,EMPSZES,PAYANN&for=zipcode:114

In [22]:
merged_df

,Year,ZIPCODE,Total Annual Payroll,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Whites Alone,Non Whites
0,2012,10001,8118354,20579.0,35.7,76138.0,74212.0,2672.0,12.984110,13654.0,6925
1,2012,10002,585960,80323.0,39.9,32265.0,25518.0,22136.0,27.558732,24852.0,55471
2,2012,10003,5244723,56614.0,32.2,91767.0,73196.0,6153.0,10.868336,43932.0,12682
3,2012,10004,9670296,2780.0,33.8,127281.0,127628.0,300.0,10.791367,2166.0,614
4,2012,10005,5947321,5724.0,28.4,120729.0,99403.0,590.0,10.307477,4238.0,1486
5,2012,10006,2130252,2696.0,31.5,115787.0,98790.0,257.0,9.532641,1871.0,825
6,2012,10007,2299621,6188.0,34.5,210125.0,156554.0,146.0,2.359405,4463.0,1725
7,2012,10009,378053,62397.0,37.0,59034.0,44705.0,12380.0,19.840697,37680.0,24717
8,2012,10010,8044200,29396.0,35.3,95556.0,84619.0,3799.0,12.923527,22061.0,7335
9,2012,10011,4399264,51853.0,40.3,101515.0,113300.0,4461.0,8.603167,40428.0,11425


In [26]:
cleaned_merged_df= merged_df.fillna('Info not available')
cleaned_merged_df

,Year,ZIPCODE,Total Annual Payroll,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Whites Alone,Non Whites
0,2012,10001,8118354,20579.0,35.7,76138,74212,2672.0,12.9841,13654.0,6925
1,2012,10002,585960,80323.0,39.9,32265,25518,22136.0,27.5587,24852.0,55471
2,2012,10003,5244723,56614.0,32.2,91767,73196,6153.0,10.8683,43932.0,12682
3,2012,10004,9670296,2780.0,33.8,127281,127628,300.0,10.7914,2166.0,614
4,2012,10005,5947321,5724.0,28.4,120729,99403,590.0,10.3075,4238.0,1486
5,2012,10006,2130252,2696.0,31.5,115787,98790,257.0,9.53264,1871.0,825
6,2012,10007,2299621,6188.0,34.5,210125,156554,146.0,2.35941,4463.0,1725
7,2012,10009,378053,62397.0,37,59034,44705,12380.0,19.8407,37680.0,24717
8,2012,10010,8044200,29396.0,35.3,95556,84619,3799.0,12.9235,22061.0,7335
9,2012,10011,4399264,51853.0,40.3,101515,113300,4461.0,8.60317,40428.0,11425


In [ ]:
# change white only column header to Caucasian only

In [27]:
cleaned_merged_df.to_csv("clean_census_Output.csv")